In [ ]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append('/home/merve/code/data-data-preparation')

👉 Dataset’inizi import edin:

In [4]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [11]:
orders.head(1)

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0.0,0.0,4.0,1,1,29.99,8.72


In [13]:
features = [
    "wait_time", "expected_wait_time", "delay_vs_expected", "number_of_items","number_of_sellers", "price", "freight_value"
]

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [16]:
#standardizasyon
orders_standardized = orders.copy()
mu = orders[features].mean()
sigma = orders[features].std()
orders_standardized[features] = (orders[features] - mu) / sigma
orders_standardized

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,-0.432084,-0.935929,-0.162119,delivered,0.0,0.0,4.0,-0.264657,-0.112588,-0.514185,-0.653042
1,53cdb2fc8bc7dce0b6741e2150273451,0.133144,-0.525643,-0.162119,delivered,0.0,0.0,4.0,-0.264657,-0.112588,-0.086552,0.000102
2,47770eb9100c2d0c44946d9cf07ec65d,-0.330823,0.330839,-0.162119,delivered,1.0,0.0,5.0,-0.264657,-0.112588,0.112056,-0.164580
3,949d5b44dbf5de918fe9c16f97b45f8a,0.072525,0.279362,-0.162119,delivered,1.0,0.0,5.0,-0.264657,-0.112588,-0.441828,0.206652
4,ad21c59c0840e6cb83a9ceb5573f8159,-1.020282,-1.327755,-0.162119,delivered,1.0,0.0,5.0,-0.264657,-0.112588,-0.562825,-0.653042
...,...,...,...,...,...,...,...,...,...,...,...,...
97002,9c5dedf39a927c1b2549525ed64a053c,-0.455195,-0.588472,-0.162119,delivered,1.0,0.0,5.0,-0.264657,-0.112588,-0.311673,-0.450214
97003,63943bddc261676b46f01ca7ac2f7bd8,1.022594,-0.032290,-0.162119,delivered,0.0,0.0,4.0,-0.264657,-0.112588,0.184365,-0.123642
97004,83c1379a015df1e13d02aae0204711ab,1.304464,0.758343,-0.162119,delivered,1.0,0.0,5.0,-0.264657,-0.112588,0.334236,1.966047
97005,11c177c8e97725db2631073c19f07b62,0.482549,1.525669,-0.162119,delivered,0.0,0.0,2.0,1.603238,-0.112588,1.076557,2.717815


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [17]:
X_standardized = orders_standardized[features]
X_standardized.head()

,wait_time,expected_wait_time,delay_vs_expected,number_of_items,number_of_sellers,price,freight_value
0,-0.432084,-0.935929,-0.162119,-0.264657,-0.112588,-0.514185,-0.653042
1,0.133144,-0.525643,-0.162119,-0.264657,-0.112588,-0.086552,0.000102
2,-0.330823,0.330839,-0.162119,-0.264657,-0.112588,0.112056,-0.164580
3,0.072525,0.279362,-0.162119,-0.264657,-0.112588,-0.441828,0.206652
4,-1.020282,-1.327755,-0.162119,-0.264657,-0.112588,-0.562825,-0.653042


In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

df = pd.DataFrame()
df["features"] = X_standardized.columns
df["vif_index"] = [vif(X_standardized.values, i) for i in range(X_standardized.shape[1])]
round(df,2)

,features,vif_index
0,wait_time,2.83
1,expected_wait_time,1.44
2,delay_vs_expected,2.38
3,number_of_items,1.35
4,number_of_sellers,1.10
5,price,1.20
6,freight_value,1.58


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [19]:
logit_one = smf.logit(formula='dim_is_one_star ~'+ ' + '.join(features), data=orders_standardized).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.273079
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                96353
Model:                          Logit   Df Residuals:                    96345
Method:                           MLE   Df Model:                            7
Date:                Sat, 03 Jan 2026   Pseudo R-squ.:                  0.1461
Time:                        11:57:39   Log-Likelihood:                -26312.
converged:                       True   LL-Null:                       -30814.
Covariance Type:            nonrobust   LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -2.4824      0.013   -189.580      0.000      -2.508      -2.457
wait_time              0.7956      0.019     41.530      0.000       0.758       0.833
expected_wait_time    -0.2484      0.016    -15.450      0.000      -0.280      -0.217
delay_vs_expected      0.1573      0.020      7.794      0.000       0.118       0.197
number_of_items        0.2645      0.011     24.977      0.000       0.244       0.285
number_of_sellers      0.1867      0.008     23.639      0.000       0.171       0.202
price                  0.0579      0.011      5.127      0.000       0.036       0.080
freight_value         -0.0422      0.013     -3.193      0.001      -0.068      -0.016
======================================================================================
"""

`Logit 5️⃣`

In [20]:
logit_five = smf.logit(formula='dim_is_five_star ~'+ ' + '.join(features), data=orders_standardized).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.636930
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                96353
Model:                          Logit   Df Residuals:                    96345
Method:                           MLE   Df Model:                            7
Date:                Sat, 03 Jan 2026   Pseudo R-squ.:                 0.05787
Time:                        11:58:15   Log-Likelihood:                -61370.
converged:                       True   LL-Null:                       -65140.
Covariance Type:            nonrobust   LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.3427      0.007     48.102      0.000       0.329       0.357
wait_time             -0.5389      0.012    -44.095      0.000      -0.563      -0.515
expected_wait_time     0.0943      0.008     11.206      0.000       0.078       0.111
delay_vs_expected     -0.3818      0.024    -16.046      0.000      -0.428      -0.335
number_of_items       -0.1447      0.008    -17.454      0.000      -0.161      -0.128
number_of_sellers     -0.1450      0.008    -18.550      0.000      -0.160      -0.130
price                  0.0167      0.008      2.201      0.028       0.002       0.032
freight_value          0.0180      0.009      2.071      0.038       0.001       0.035
======================================================================================
"""

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [21]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [a]

🧪 __Kodunu Test Et__

In [22]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/merve/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/merve/code/data-logit/tests
plugins: dash-3.3.0, typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!